In [1]:
import os
import zipfile
import requests

# Завантаження файлу
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
filename = "kagglecatsanddogs_5340.zip"

if not os.path.exists(filename):
    print("Завантаження датасету...")
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print("Завантаження завершено!")

# Розпакування архіву
if not os.path.exists("cats_and_dogs_dataset"):
    print("Розпакування архіву...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall("cats_and_dogs_dataset")
    print("Розпакування завершено!")
else:
    print("Датасет вже розпакований.")

Завантаження датасету...
Завантаження завершено!
Розпакування архіву...
Розпакування завершено!


In [2]:
import os
import shutil

# Визначимо кореневий каталог
root_dir = '/tmp/cats-v-dogs'

# Очистимо каталог, щоб уникнути помилки FileExistsError у випадку багаторазового запуску функції
if os.path.exists(root_dir):
    shutil.rmtree(root_dir)

# Оголосимо функцію для створення каталогів для навчання та валідації
def create_train_val_dirs(root_path):
    """
    Створює каталоги для навчання та валідації

    Args:
        root_path (string) - базовий шлях каталогу для створення підкаталогів

    Returns:
        None
    """
    
    ### ПОЧАТОК КОДУ

    # Використовуйте os.makedirs для створення каталогів з проміжними підкаталогами
    # Не захардкодьте шляхи. Використовуйте os.path.join для додавання нових каталогів до параметра root_path

    os.makedirs(root_path)
    os.makedirs(os.path.join(root_path, 'training', 'cats'))
    os.makedirs(os.path.join(root_path, 'training', 'dogs'))
    os.makedirs(os.path.join(root_path, 'validation', 'cats'))
    os.makedirs(os.path.join(root_path, 'validation', 'dogs'))

    ### КІНЕЦЬ КОДУ

# Спробуємо викликати функцію для створення каталогів
try:
    create_train_val_dirs(root_path=root_dir)
except FileExistsError:
    print("Ти не повинен бачити це, оскільки верхній каталог видаляється перед виконанням")
    
# Перевіримо створені каталоги
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/tmp/cats-v-dogs\training
/tmp/cats-v-dogs\validation
/tmp/cats-v-dogs\training\cats
/tmp/cats-v-dogs\training\dogs
/tmp/cats-v-dogs\validation\cats
/tmp/cats-v-dogs\validation\dogs


In [5]:
import os
import random
from shutil import copyfile

# Визначаємо шляхи до директорій для котів та собак
CAT_SOURCE_DIR = "cats_and_dogs_dataset/PetImages/Cat/"
DOG_SOURCE_DIR = "cats_and_dogs_dataset/PetImages/Dog/"

TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
VALIDATION_CATS_DIR = "/tmp/cats-v-dogs/validation/cats/"

TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
VALIDATION_DOGS_DIR = "/tmp/cats-v-dogs/validation/dogs/"

# Визначаємо пропорцію розділу даних
split_size = 0.9

# Оголошуємо функцію split_data
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
    """
    Розділяє дані на навчальний та валідаційний набори

    Args:
        SOURCE_DIR (string): шлях до директорії з зображеннями
        TRAINING_DIR (string): шлях до директорії для навчального набору
        VALIDATION_DIR (string): шлях до директорії для валідаційного набору
        SPLIT_SIZE (float): пропорція набору даних, що використовується для навчання

    Returns:
        None
    """
    # Отримуємо список файлів у вихідній директорії
    files = os.listdir(SOURCE_DIR)
    # Перемішуємо список
    random.shuffle(files)
    # Визначаємо розмір навчального набору
    split_index = int(len(files) * SPLIT_SIZE)
    # Розділяємо дані
    training_files = files[:split_index]
    validation_files = files[split_index:]
    
    # Копіюємо зображення для навчання
    for file in training_files:
        source = os.path.join(SOURCE_DIR, file)
        destination = os.path.join(TRAINING_DIR, file)
        # Перевіряємо, чи файл не має нульової довжини перед копіюванням
        if os.path.getsize(source) > 0:
            copyfile(source, destination)
        else:
            print(f"{file} має нульову довжину, тому ігнорується.")

    # Копіюємо зображення для валідації
    for file in validation_files:
        source = os.path.join(SOURCE_DIR, file)
        destination = os.path.join(VALIDATION_DIR, file)
        # Перевіряємо, чи файл не має нульової довжини перед копіюванням
        if os.path.getsize(source) > 0:
            copyfile(source, destination)
        else:
            print(f"{file} має нульову довжину, тому ігнорується.")

# Викликаємо функцію split_data для котів
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
# Викликаємо функцію split_data для собак
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

# Виведемо інформацію про кількість зображень у кожному наборі
print(f"\n\nОригінальна директорія з котами містить {len(os.listdir(CAT_SOURCE_DIR))} зображень")
print(f"Оригінальна директорія з собаками містить {len(os.listdir(DOG_SOURCE_DIR))} зображень\n")

print(f"У навчальному наборі для котів - {len(os.listdir(TRAINING_CATS_DIR))} зображень")
print(f"У навчальному наборі для собак - {len(os.listdir(TRAINING_DOGS_DIR))} зображень")
print(f"У валідаційному наборі для котів - {len(os.listdir(VALIDATION_CATS_DIR))} зображень")
print(f"У валідаційному наборі для собак - {len(os.listdir(VALIDATION_DOGS_DIR))} зображень")

666.jpg має нульову довжину, тому ігнорується.
11702.jpg має нульову довжину, тому ігнорується.


Оригінальна директорія з котами містить 12501 зображень
Оригінальна директорія з собаками містить 12501 зображень

У навчальному наборі для котів - 11249 зображень
У навчальному наборі для собак - 11249 зображень
У валідаційному наборі для котів - 1251 зображень
У валідаційному наборі для собак - 1251 зображень


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = '/tmp/cats-v-dogs/training/'
VALIDATION_DIR = '/tmp/cats-v-dogs/validation/'

# Оголошуємо функцію train_val_generators
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    """
    Створює генератори даних для навчання та валідації

    Args:
        TRAINING_DIR (string): шлях до директорії з зображеннями для навчання
        VALIDATION_DIR (string): шлях до директорії з зображеннями для валідації

    Returns:
        train_generator, validation_generator - кортеж, що містить генератори
    """
    # Інстанціюємо клас ImageDataGenerator для навчальних даних (не забудьте встановити аргументи для аугментації зображень)
    train_datagen = ImageDataGenerator(rescale=1./255)

    # Передаємо потрібні аргументи методу flow_from_directory
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=32,
                                                        class_mode='binary',
                                                        target_size=(150, 150))

    # Інстанціюємо клас ImageDataGenerator для валідаційних даних (не забудьте встановити аргумент rescale)
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Передаємо потрібні аргументи методу flow_from_directory
    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                  batch_size=32,
                                                                  class_mode='binary',
                                                                  target_size=(150, 150))
    
    return train_generator, validation_generator

# Тестуємо генератори
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 22496 images belonging to 2 classes.
Found 2502 images belonging to 2 classes.


In [ ]:
import tensorflow as tf

# Оголошуємо функцію create_model
def create_model():
    # Визначаємо модель Sequential
    model = tf.keras.models.Sequential([
        # Перший згортковий шар
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        # Другий згортковий шар
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Третій згортковий шар
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Перетворення 3D-даних у 1D
        tf.keras.layers.Flatten(),
        # Повністю з'єднаний шар
        tf.keras.layers.Dense(512, activation='relu'),
        # Вихідний шар
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Компілюємо модель
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Отримуємо ненавчену модель
model = create_model()

# Навчаємо модель
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

In [ ]:
import matplotlib.pyplot as plt

# Отримуємо точність та втрати на навчанні та валідації
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))  # Кількість епох

# Графік точності навчання та валідації
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

# Графік втрат навчання та валідації
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

def download_history():
    import pickle
    from google.colab import files

    with open('history.pkl', 'wb') as f:
        pickle.dump(history.history, f)

    files.download('history.pkl')

download_history()